In [ ]:
import zarrita
import os

In [ ]:
root_path = "s3://carbonplan-benchmarks/data"
orig_path = "NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1f1/tasmax/tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn"

In [ ]:
source_stores = [
    "pyramids-v2-3857-0",
    "pyramids-v2-3857-1",
    "pyramids-v2-4326-0",
]

groups = ['0', '1', '2', '3']
arrays = ['tasmax', 'time', 'x', 'y']

In [ ]:
def convert_array_to_v3(*, group: str, array: str, source_store: str, target_store: str):
    source_array = zarrita.ArrayV2.open(source_store / group / array)
    data = source_array[:]
    attrs = source_array.attributes
    chunks = source_array.metadata.chunks
    target_array = zarrita.Array.create(
        target_store / group / array,
        shape=data.shape,
        dtype=data.dtype,
        chunk_shape=chunks,
        attributes=attrs,
        codecs=[zarrita.codecs.gzip_codec()],
    )
    target_array[:] = data

In [ ]:
def convert_array_to_sharded_v3(*, group: str, array: str, source_store: str, target_store: str):
    source_array = zarrita.ArrayV2.open(source_store / group / array)
    data = source_array[:]
    attrs = source_array.attributes
    chunks = source_array.metadata.chunks
    target_array = zarrita.Array.create(
        target_store / group / array,
        shape=data.shape,
        dtype=data.dtype,
        chunk_shape=data.shape,
        attributes=attrs,
        codecs=[
            zarrita.codecs.sharding_codec(
                chunk_shape=chunks,
                codecs=[zarrita.codecs.gzip_codec()],
            )
        ],
    )
    target_array[:] = data

In [ ]:
def convert_group_to_v3(*, group: str, source_store: str, target_store: str):
    source_group = zarrita.GroupV2.open(source_store / group)
    attrs = source_group.attributes
    zarrita.Group.create(target_store / group, attributes=attrs)

In [ ]:
def convert_store_to_v3(*, source_store, target_store, sharded: bool = False):
    for group in groups:
        convert_group_to_v3(group=group, source_store=source_store, target_store=target_store)
        for array in arrays:
            if sharded:
                convert_array_to_sharded_v3(
                    group=group, array=array, source_store=source_store, target_store=target_store
                )
            else:
                convert_array_to_v3(
                    group=group, array=array, source_store=source_store, target_store=target_store
                )

### Zarr v3 (no shards)

In [ ]:
for path in source_stores:
    orig = os.path.join(root_path, orig_path, path)
    target = orig.replace("v2", "v3")
    print(orig)
    print(target)
    source_store = zarrita.RemoteStore(orig)
    target_store = zarrita.RemoteStore(target)
    source_group = zarrita.GroupV2.open(source_store)
    attrs = source_group.attributes
    zarrita.Group.create(target_store, attributes=attrs)
    convert_store_to_v3(source_store=source_store, target_store=target_store, sharded=False)

### Zarr v3 (sharded)

In [ ]:
for path in source_stores:
    orig = os.path.join(root_path, orig_path, path)
    target = orig.replace("v2", "v3-sharded")
    print(orig)
    print(target)
    source_store = zarrita.RemoteStore(orig)
    target_store = zarrita.RemoteStore(target)
    source_group = zarrita.GroupV2.open(source_store)
    attrs = source_group.attributes
    zarrita.Group.create(target_store, attributes=attrs)
    convert_store_to_v3(source_store=source_store, target_store=target_store, sharded=True)